In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Q1. Outlier가 데이터 셋에 exist할 경우 선형회귀에 대한 성능을 compare해보세요


# 데이터들의 의미
(https://www.kaggle.com/c/nyc-taxi-trip-duration/data)

id - a unique identifier for each trip

vendor_id - a code indicating the provider associated with the trip record

pickup_datetime - date and time when the meter was engaged

dropoff_datetime - date and time when the meter was disengaged

passenger_count - the number of passengers in the vehicle (driver entered value)

pickup_longitude - the longitude where the meter was engaged

pickup_latitude - the latitude where the meter was engaged

dropoff_longitude - the longitude where the meter was disengaged

dropoff_latitude - the latitude where the meter was disengaged

store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip



# target value: trip_duration - duration of the trip in seconds

In [3]:
data_df = pd.read_csv('train.csv')

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


모든 feature가 수치형 데이터임이 아님을 알 수있다.

id, vendor_id, store_and_fwd_flag에 대한 처리가 필요!

In [5]:
# index가 id역할을 하니까 id는 드랍함
data_df.drop('id', axis=1, inplace=True)

In [6]:
np.round(data_df.describe(),2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,1.66,-73.97,40.75,-73.97,40.75,959.49
std,0.50,1.31,0.07,0.03,0.07,0.04,5237.43
min,1.00,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,9.00,-61.34,51.88,-61.34,43.92,3526282.00


min, quantile, max를 살펴보면 어떤 outlier가 trip_duration에 있음을 알 수있습니다.

vendor id, id는 각각 identify를 위한 데이터이므로 drop을 사용하여 처리합니다. pick up time, drop off time은 각각 시계열 데이터인데, 이 책에서 시계열 처리에 대해 다루지 않았으므로 drop을 해줍니다.

In [7]:
data_df.drop(['pickup_datetime', 'dropoff_datetime'], axis = 1, inplace = True)

In [8]:
data_df.head(3)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,N,2124


store_and_fwd_flag와 trip_duration의 상관관계를 알아본다. 만약 상관관계가 적다면 drop하고 그렇지 않다면 핸들링을 따로 한다

In [9]:
sff_cat = {'N':0, 'Y':1}
cat_fun = lambda x:sff_cat[x]
sff = data_df['store_and_fwd_flag'].map(cat_fun)
data_df.groupby('store_and_fwd_flag')['trip_duration'].mean()

store_and_fwd_flag
N     958.819706
Y    1080.763331
Name: trip_duration, dtype: float64

위의 결과를 보고 store_and_fwd_flag을 drop할지 하지 않을지 생각해보세요

In [10]:
# N일 떄의 평균, Y일 때의 평균이 큰 차이가 없으므로 drop한다
data_df.drop('store_and_fwd_flag', axis=1, inplace=True)

In [11]:
data_df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,2124
3,2,1,-74.010040,40.719971,-74.012268,40.706718,429
4,2,1,-73.973053,40.793209,-73.972923,40.782520,435


trip_duration에 outlier 수를 파악해보고, 이 outlier들을 drop했을 때와 안했을 때의 성능 수준을 선형 회귀 모델들로 비교해 보겠습니다.

In [12]:
np.sum(data_df.trip_duration > 10000)

2123

# outlier가 있는 데이터셋을 사용하여 선형 회귀 모델 평가.

In [13]:
#outlier를 제거하지 않은 데이터 셋을 훈련세트와 데이터 세트로 나눈다.
X = data_df.drop('trip_duration', axis = 1)
y = data_df.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

LinearRegression을 사용하여 train set을 학습시킨후 test set으로 평가해보세요

In [14]:
def get_linear_result(X_train, y_train):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_preds = lr.predict(X_test)

    mse = mean_squared_error(y_test, y_preds)
    
    print ("MSE: {0: 3f}".format(mse))

In [15]:
get_linear_result(X_train, y_train)

MSE:  10602333.630458


Pipeline과 PolynomialFeatures를 가지고 다항회귀를 사용해서 평가해보세요

In [18]:
def polynomial_func(X):
    y = 1 + 2 * X + X**2 + X**3 + X**4
    return y

def get_polynomial_result(X_train, y_train):
    model = Pipeline([('poly', PolynomialFeatures(degree=3)), ('linear', LinearRegression())])
    X = np.arange(4).reshape(2, 2)
    y = polynomial_func(X)
    
    model.fit (X_train, y_train)
    
    mse = -1*np.mean(cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=10))
    
    print ("MSE: {0: 3f}".format(mse))

In [19]:
get_polynomial_result(X_train, y_train)

MSE:  304468828927.748840


Ridge 모델 학습

다양한 learning rate 가지고 ridge 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [27]:
def get_linear_reg_eval(model_name, params=None, X_train_n=None, y_train_n=None, verbose=True):
    coeff_df = pd.DataFrame()
    if verbose : print('####### ', model_name , '#######')
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_train_n, 
                                             y_train_n, scoring="neg_mean_squared_error", cv = 5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {0}일 때 5 폴드 세트의 평균 RMSE: {1:.3f} '.format(param, avg_rmse))
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        model.fit(X_train , y_train)
        # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가. 
        coeff = pd.Series(data=model.coef_ , index=X_train.columns )
        colname='alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df

In [41]:
ridge_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_ridge_df =get_linear_reg_eval('Ridge', params=ridge_alphas, X_train_n=X_train, y_train_n=y_train)

#######  Ridge #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 658.706 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 658.705 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 658.693 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 658.678 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 658.620 


In [42]:
sort_column = 'alpha:'+str(ridge_alphas[0])
coeff_ridge_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,2507.694836,2507.659007,2507.181418,2506.584777,2504.202031
passenger_count,6.280951,6.280964,6.281129,6.281335,6.282160
vendor_id,4.347017,4.347028,4.347178,4.347364,4.348109
dropoff_longitude,-353.669988,-353.643906,-353.296263,-352.862037,-351.128752
dropoff_latitude,-2039.771814,-2039.753097,-2039.503478,-2039.191299,-2037.940870
pickup_latitude,-2839.184194,-2839.125343,-2838.340954,-2837.361251,-2833.451117


Lasso 모델 학습

다양한 learning rate 가지고 Lasso 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [28]:
lasso_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_lasso_df =get_linear_reg_eval('Lasso', params=lasso_alphas, X_train_n=X_train, y_train_n=y_train)

#######  Lasso #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 5261.273 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 5261.267 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 5261.233 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 5261.258 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 5261.682 


In [29]:
sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,2491.720754,2464.898372,2110.246524,1667.037523,1214.515113
vendor_id,194.980165,194.875548,193.477083,191.728875,183.677383
passenger_count,8.457462,8.458743,8.475702,8.496896,8.515973
dropoff_longitude,-833.104452,-807.789173,-472.721795,-53.975267,0.000000
dropoff_latitude,-1637.328802,-1620.862112,-1401.929438,-1128.285705,-79.015962
pickup_latitude,-3703.546057,-3690.474628,-3514.930603,-3295.455935,-2047.364139


ElasticNet 모델 학습

다양한 learning rate 가지고 ElasticNet 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [31]:
elastic_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df =get_linear_reg_eval('ElasticNet', params=elastic_alphas,
                                      X_train_n=X_train, y_train_n=y_train)

#######  ElasticNet #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 5263.291 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 5263.550 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 5264.377 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 5264.660 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 5265.074 


In [32]:
sort_column = 'alpha:'+str(elastic_alphas[0])
coeff_elastic_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,349.319165,264.463070,61.836766,30.653773,8.881339
vendor_id,179.354220,173.131137,118.247412,84.704083,39.475902
dropoff_longitude,140.585309,115.499159,30.854610,15.066686,3.657879
passenger_count,10.899516,11.519133,16.170308,17.913908,16.520244
dropoff_latitude,-175.228703,-124.299177,-23.726603,-10.724851,-2.023317
pickup_latitude,-232.580062,-165.024638,-32.168174,-14.974073,-3.448184


# outlier를 없엔 데이터 셋을 사용하여 선형 회귀모델 평가.

In [33]:
outlier_idx = y[y > 10000].index
data_dropped_outlier = data_df.drop(outlier_idx, axis = 0)

In [34]:
#데이터가 2000개 가량 줄어든 것을 확인할 수 있다.
data_dropped_outlier.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456521 entries, 0 to 1458643
Data columns (total 7 columns):
vendor_id            1456521 non-null int64
passenger_count      1456521 non-null int64
pickup_longitude     1456521 non-null float64
pickup_latitude      1456521 non-null float64
dropoff_longitude    1456521 non-null float64
dropoff_latitude     1456521 non-null float64
trip_duration        1456521 non-null int64
dtypes: float64(4), int64(3)
memory usage: 88.9 MB


In [35]:
X_dropped = data_dropped_outlier.drop('trip_duration', axis = 1)
y_dropped = data_dropped_outlier.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, random_state = 42)

앞서서 했던 outlier를 제거하기 전 시행했던 각각의 모델들을 위의 데이터셋을 통해 평가해보세요.

In [43]:
print ("###### Linear Regression ######")
get_linear_result(X_train, y_train)

print("\n###### Polynomial Regression ######")
get_polynomial_result(X_train, y_train)

print (" ")
ridge_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_ridge_df =get_linear_reg_eval('Ridge', params=ridge_alphas, X_train_n=X_train, y_train_n=y_train)

print (" ")
lasso_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_lasso_df =get_linear_reg_eval('Lasso', params=lasso_alphas, X_train_n=X_train, y_train_n=y_train)

print (" ")
elastic_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df =get_linear_reg_eval('ElasticNet', params=elastic_alphas,
                                      X_train_n=X_train, y_train_n=y_train)

###### Linear Regression ######
MSE:  405067.949520

###### Polynomial Regression ######
MSE:  102533223693.254425
 
#######  Ridge #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 658.706 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 658.705 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 658.693 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 658.678 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 658.620 
 
#######  Lasso #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 658.053 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 657.780 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 654.550 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 651.458 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 647.926 
 
#######  ElasticNet #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 650.015 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 651.758 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 656.088 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 656.760 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 657.240 


# Q2. 이 데이터 셋에서는 규제가 그리 효과적이지 못한 것 처럼 보입니다. 그 이유는 무엇일까요?



# Q3. 이 데이터셋은 우리가 원하는 trip duration을 구하기에는 연관이 적은 것 같습니다. 데이터를 가공한다면, 어떤식으로 하면 trip duration과 관련이 있을까요?

# 부록:파이썬의 시계열 데이터 

파이썬 라이브러리에는 날짜와 시간을 위한 자료형과 달력과 관련된 기능을 제공하는 자료형이 있습니다..

파이썬 기본 자료형: datetime, time, calender

date: 그레고리언 달력을 사용하여 년, 월, 일을 저장한다.

time: 하루 중 시간을 시간, 분, 초, 마이크로 초 단위로 저장한다.

datetime: 날짜와 시간을 같이 저장한다

timedelta: 두 datetime 값 간의 차이(일, 초, 마이크로초)를 표현한다

In [ ]:
trip_df = pd.read_csv('./train.csv')
trip_df.drop(['id', 'vendor_id'], axis= 1,inplace = True)

간단한 시계열 포멧

%Y: 네자리 연도

%y: 두자리 연도

%m: 두자리 월 ex:01 은 1월

%d: 두자리 일 ex:03은 3일

%H: 24시간 형식 시간

%I: 12시간 형식 시간

%M: 두 자리 분

%S: 초

In [ ]:
from datetime import datetime
# series에 있는 시계열 데이터는 현재 문자열 형태로 저장되어 있기 때문에 처리가 용이한 datetime 타입으로 바꾼다.
d2dt = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') #datetime형으로 변환할 때 시계열 데이터의 형태를 맞춰서 넣어줘야한다.
pickup = trip_df.pickup_datetime.map(d2dt)
dropoff = trip_df.dropoff_datetime.map(d2dt)

In [ ]:
#pickup 시간과 dropoff 시간의 간격을 초 단위로 바꾼다.
# datetime 형을 빼거나 더하면 timedelta 타입으로 바뀐다. timedelta 형은 시계열 형 데이터의 기간을 표현하는데 적합하다.
timegap = dropoff - pickup
tomsec = lambda x: x.seconds
timegap_sec = timegap.map(tosec)

In [ ]:
#timedelta형임을 확인할 수 있다.
type(timegap[0])

In [ ]:
timegap_sec

In [ ]:
#손님을 태운 시간과 내려준 시간의 차, 즉 trip_duration이 실제로는 이 데이터셋으로부터 바로 구할 수있다.
#위 데이터셋에서 trip_duration과 timegap_sec가 맞지 않는 4개의 데이터셋이 있는데 이는 시간이 잘못 기록되었음을 유추할 수 있다.
np.sum(timegap_sec != trip_df.trip_duration)